SVM
Feature Extraction Method: GLCM
Features Taken with distance = 1,3,5, angle = 0,45,90,135,180,225,270,315) (total 120 features)

In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/"MyDrive/Machine Learning/GLCM_Features"

/content/drive/MyDrive/Machine Learning/GLCM_Features


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import glob
import cv2
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn import svm
from sklearn.metrics import classification_report, confusion_matrix
import itertools
from skimage.filters import sobel
from skimage.feature import greycomatrix, greycoprops
import time

In [ ]:
X_train = list()
X_test = list()
y_train = list()
y_test = list()

# for k fold cross validation
x_new = list()
y_new = list()

In [ ]:
# 0 = crazing 
# 1 = inclusion
# 2 = patches
# 3 = pitted_surface
# 4 = rolled in scale
# 5 = scratches

In [ ]:
dataset1 = pd.read_csv('GLCM_train.csv')
print(dataset1)

      Unnamed: 0   Energy0     Corr0  Diss_sim0  Homogen0   Contrast0  \
0              0  0.014224  0.725731  15.456080  0.063373  383.715930   
1              0  0.015558  0.778272  12.804573  0.076365  263.964171   
2              0  0.015120  0.746263  13.894296  0.069790  310.096910   
3              0  0.016114  0.719193  13.510452  0.071717  292.011709   
4              0  0.013543  0.787991  14.764347  0.066498  351.571382   
...          ...       ...       ...        ...       ...         ...   
1435           0  0.033905  0.971425   3.316633  0.292426   22.962261   
1436           0  0.058558  0.959366   2.681734  0.485881   48.285804   
1437           0  0.047101  0.983943   2.665603  0.434930   30.869422   
1438           0  0.052825  0.987204   3.837789  0.358781   61.731910   
1439           0  0.052105  0.966701   3.046206  0.392387   38.874296   

       Energy1     Corr1  Diss_sim1  Homogen1  ...  Energy22    Corr22  \
0     0.013450  0.616408  18.258428  0.054306  ..

In [ ]:
dataset1.describe()

,Unnamed: 0,Energy0,Corr0,Diss_sim0,Homogen0,Contrast0,Energy1,Corr1,Diss_sim1,Homogen1,...,Energy22,Corr22,Diss_sim22,Homogen22,Contrast22,Energy23,Corr23,Diss_sim23,Homogen23,Contrast23
count,1440.0,1440.000000,1440.000000,1440.000000,1440.000000,1440.000000,1440.000000,1440.000000,1440.000000,1440.000000,...,1440.000000,1440.000000,1440.000000,1440.000000,1440.000000,1440.000000,1440.000000,1440.000000,1440.000000,1440.000000
mean,0.0,0.039076,0.914974,6.997952,0.227607,121.605174,0.036137,0.876810,8.314382,0.195434,...,0.032516,0.772890,11.269470,0.157613,316.573793,0.030477,0.711276,12.690073,0.138156,399.945973
std,0.0,0.031993,0.073490,4.495580,0.143985,123.116106,0.030687,0.108568,5.206155,0.123702,...,0.028739,0.210032,7.254543,0.107482,325.639194,0.027271,0.203431,7.335945,0.088290,373.605586
min,0.0,0.010988,0.667458,0.750251,0.059618,1.303920,0.010379,0.562416,1.171486,0.050095,...,0.009259,0.206686,1.393538,0.035860,3.437795,0.009004,0.146048,1.785064,0.033643,6.240551
25%,0.0,0.017849,0.873547,2.855704,0.104752,19.534177,0.016751,0.807872,3.437382,0.092006,...,0.014914,0.626166,4.306244,0.070809,41.410128,0.014451,0.567816,6.164072,0.066144,99.654688
50%,0.0,0.028030,0.944733,6.217249,0.163017,70.343480,0.025727,0.925800,7.613924,0.139287,...,0.022618,0.873883,10.672500,0.108209,210.457744,0.022242,0.776509,11.322847,0.103373,257.521658
75%,0.0,0.053751,0.971263,10.763901,0.342913,205.148222,0.048970,0.961424,12.498498,0.293971,...,0.043850,0.939595,16.978769,0.243213,509.245404,0.039847,0.876135,18.615369,0.203037,645.969817
max,0.0,0.347222,0.997559,16.727513,0.675018,450.635955,0.346124,0.996641,20.197798,0.560829,...,0.344011,0.995447,28.120949,0.504314,1419.477231,0.336819,0.994030,30.111594,0.455078,1752.488468


In [ ]:
x = dataset1.iloc[:,1:].values
print(len(x[0]))

120


In [ ]:
print(x)

[[1.42235469e-02 7.25731266e-01 1.54560804e+01 ... 2.50577884e+01
  4.05208138e-02 1.01923751e+03]
 [1.55583846e-02 7.78271956e-01 1.28045729e+01 ... 1.96631352e+01
  5.13562318e-02 6.26654545e+02]
 [1.51195333e-02 7.46262736e-01 1.38942965e+01 ... 2.22071793e+01
  4.62140829e-02 7.96455565e+02]
 ...
 [4.71007360e-02 9.83943252e-01 2.66560302e+00 ... 7.39074865e+00
  2.49918243e-01 2.54045684e+02]
 [5.28245045e-02 9.87203605e-01 3.83778894e+00 ... 1.07429456e+01
  1.96037798e-01 5.11558647e+02]
 [5.21048101e-02 9.66701159e-01 3.04620603e+00 ... 6.07725948e+00
  2.54339847e-01 1.77470117e+02]]


In [ ]:
print(x[0])

[1.42235469e-02 7.25731266e-01 1.54560804e+01 6.33728906e-02
 3.83715930e+02 1.34496351e-02 6.16407720e-01 1.82584278e+01
 5.43057083e-02 5.36650034e+02 1.45606429e-02 7.59055851e-01
 1.45129397e+01 6.73377845e-02 3.37119020e+02 1.36664513e-02
 6.55656905e-01 1.73407742e+01 5.77192005e-02 4.81738744e+02
 1.42235469e-02 7.25731266e-01 1.54560804e+01 6.33728906e-02
 3.83715930e+02 1.34496351e-02 6.16407720e-01 1.82584278e+01
 5.43057083e-02 5.36650034e+02 1.45606429e-02 7.59055851e-01
 1.45129397e+01 6.73377845e-02 3.37119020e+02 1.36664513e-02
 6.55656905e-01 1.73407742e+01 5.77192005e-02 4.81738744e+02
 1.26830436e-02 3.95794743e-01 2.26954061e+01 4.46945403e-02
 8.42411802e+02 1.26403998e-02 3.88674657e-01 2.28848332e+01
 4.50317227e-02 8.54238369e+02 1.26798942e-02 4.15270877e-01
 2.25397208e+01 4.35298264e-02 8.18826878e+02 1.27552290e-02
 4.36699820e-01 2.20984083e+01 4.46882495e-02 7.87138557e+02
 1.26830436e-02 3.95794743e-01 2.26954061e+01 4.46945403e-02
 8.42411802e+02 1.264039

In [ ]:
print(len(x))

1440


In [ ]:
y = list()

In [ ]:
for i in range(0,6) :
    for j in range(0,240):
        y.append(i)


In [ ]:
print(y)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [ ]:
X_train = x
y_train = y

In [ ]:
dataset1 = pd.read_csv('GLCM_test.csv')
print(dataset1)

     Unnamed: 0   Energy0     Corr0  Diss_sim0  Homogen0   Contrast0  \
0             0  0.014085  0.748947  15.308317  0.063230  375.439874   
1             0  0.013354  0.812510  14.557965  0.068531  341.479372   
2             0  0.013476  0.759687  15.553065  0.063098  390.181206   
3             0  0.013039  0.771689  15.981709  0.062617  409.665628   
4             0  0.013783  0.757698  15.442638  0.064622  387.517764   
..          ...       ...       ...        ...       ...         ...   
355           0  0.062155  0.995837   1.575452  0.488198    5.920075   
356           0  0.067211  0.985235   1.432513  0.498722    4.875427   
357           0  0.055791  0.988033   2.337362  0.355444   10.801583   
358           0  0.046354  0.989063   2.493492  0.373817   19.243593   
359           0  0.055859  0.979090   2.404749  0.356955   14.151985   

      Energy1     Corr1  Diss_sim1  Homogen1  ...  Energy22    Corr22  \
0    0.013353  0.651627  17.977299  0.054884  ...  0.012267  0

In [ ]:
x = dataset1.iloc[:,1:].values
print(len(x[0]))

120


In [ ]:
print(x)
print(x[0])

[[1.40851053e-02 7.48946716e-01 1.53083166e+01 ... 2.52919877e+01
  4.12090470e-02 1.05380630e+03]
 [1.33542182e-02 8.12510406e-01 1.45579648e+01 ... 2.30294409e+01
  4.41729560e-02 8.60134241e+02]
 [1.34756312e-02 7.59686590e-01 1.55530653e+01 ... 2.56813307e+01
  3.83698324e-02 1.07731544e+03]
 ...
 [5.57906864e-02 9.88033099e-01 2.33736181e+00 ... 9.74463765e+00
  1.72121360e-01 4.95202312e+02]
 [4.63544075e-02 9.89062761e-01 2.49349246e+00 ... 1.14735267e+01
  1.89257210e-01 7.34857429e+02]
 [5.58592725e-02 9.79089631e-01 2.40474874e+00 ... 9.31156289e+00
  1.68402502e-01 3.99055940e+02]]
[1.40851053e-02 7.48946716e-01 1.53083166e+01 6.32298821e-02
 3.75439874e+02 1.33527101e-02 6.51626626e-01 1.79772986e+01
 5.48840623e-02 5.20755663e+02 1.44900611e-02 7.84241722e-01
 1.41835678e+01 6.95831467e-02 3.22941960e+02 1.36216790e-02
 6.93382163e-01 1.68698770e+01 5.94963105e-02 4.58334588e+02
 1.40851053e-02 7.48946716e-01 1.53083166e+01 6.32298821e-02
 3.75439874e+02 1.33527101e-02 6.5

In [ ]:
print(len(x))

360


In [ ]:
y = list()

In [ ]:
for i in range(0,6) :
    for j in range(0,60):
        y.append(i)


In [ ]:
X_test = x
y_test = y

In [ ]:
print(len(X_train))
print(len(X_test))
print(len(y_train))
print(len(y_test))

1440
360
1440
360


In [ ]:
X_train=np.array(X_train)
y_train=np.array(y_train)
X_test=np.array(X_test)
y_test=np.array(y_test)

In [ ]:
print ('Train set:', X_train.shape,  y_train.shape)
print ('Test set:', X_test.shape,  y_test.shape)

Train set: (1440, 120) (1440,)
Test set: (360, 120) (360,)


In [ ]:
nsamples, nx = X_train.shape
X_train = X_train.reshape((nsamples,nx))

In [ ]:
print(X_train.shape)

(1440, 120)


In [ ]:
nsamples, nx = X_test.shape
X_test = X_test.reshape((nsamples,nx))

In [ ]:
print(X_test.shape)

(360, 120)


In [ ]:
train_begin = time.time()
clf = svm.SVC(kernel='linear')
clf.fit(X_train, y_train) 
train_end = time.time()
train_time = train_end - train_begin



In [ ]:
test_begin = time.time()
yhat = clf.predict(X_test)
test_end = time.time()
test_time = test_end - test_begin
print(yhat)

print(metrics.accuracy_score(y_test, yhat))
print(confusion_matrix(y_test, yhat))
print(classification_report(y_test, yhat))
print(train_time)
print(test_time)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 2 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 2 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 5 1 1 1 1 1 1 5 1 1 1 5 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 5 1 1
 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 0 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 0 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 0 2 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 0 3 3 3 3 3 3 3 3 0
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5
 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5]
0.9583333333333334
[[55  0  5  0  0  0]
 [ 0 56  0  0  0  4]
 [ 3  0 57  0  0  0]
 [ 2  0  0 58  0  0]
 [ 0  0  0  0 60  0]
 [ 0  0  0  0  1 59]]
              precision    recall  f1-score   support

           0       0.92      0.92      0.92        60
           1  

In [ ]:
print(train_time)
print(test_time/len(X_test))

0.08941173553466797
9.393029742770726e-06


In [ ]:
clf = svm.SVC(kernel='rbf', gamma="scale")
clf.fit(X_train, y_train) 
yhat = clf.predict(X_test)
print(yhat)

print(metrics.accuracy_score(y_test, yhat))
print(confusion_matrix(y_test, yhat))
print(classification_report(y_test, yhat))

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 5 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 2 0 2 0 2 2 2 2 2 3 2 0 2 0 2 2 3 2 2 3 0 2 2 4 2 3 2 2
 2 2 2 0 0 2 2 2 0 2 0 2 0 0 0 2 2 2 2 0 2 2 0 3 2 2 2 2 3 0 3 0 0 3 1 3 3
 3 3 3 3 0 3 1 0 3 3 3 3 3 0 1 0 3 2 3 0 1 3 1 0 3 1 3 0 3 3 3 3 0 3 3 3 0
 3 0 3 3 3 3 3 3 3 3 1 3 3 3 1 3 1 3 4 4 3 4 4 4 4 4 4 4 4 4 4 4 4 4 3 3 4
 4 4 4 4 3 4 4 4 4 4 4 4 4 3 4 4 4 4 4 4 3 3 4 3 4 4 3 3 3 4 4 3 4 4 4 4 4
 4 4 4 4 0 0 3 3 4 4 4 4 4 4 3 1 4 3 0 0 3 5 0 3 0 1 4 4 4 4 4 1 4 5 3 0 3
 4 4 4 0 4 3 0 4 1 3 4 4 4 4 4 3 3 5 0 4 4 3 4 3 4 0 4]
0.6777777777777778
[[59  0  1  0  0  0]
 [ 0 59  0  0  0  1]
 [16  0 36  7  1  0]
 [11  9  1 39  0  0]
 [ 0  0  0 12 48  0]
 [11  4  0 14 28  3]]
              precision    recall  f1-score   support

           0       0.61      0.98      0.75        60
           1  

In [ ]:
clf = svm.SVC(kernel='rbf', gamma="auto")
clf.fit(X_train, y_train) 
yhat = clf.predict(X_test)
print(yhat)

print(metrics.accuracy_score(y_test, yhat))
print(confusion_matrix(y_test, yhat))
print(classification_report(y_test, yhat))

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 3 1 1 1 1 1 1 1 1 1
 0 5 1 1 1 1 1 1 0 1 1 1 5 5 1 1 1 1 1 1 0 1 1 1 1 0 1 1 1 1 1 1 3 1 0 3 1
 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 2 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 3 0 0 0 0 0 0 0 0 0 0 0 0 0 3 0 3 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 3 3 0 0 0 3 0 3 0 4 4 4 0 4 0 4 4 0 0 0 0 4 0 4 0 4 4 4
 4 4 0 4 4 4 4 0 4 0 0 0 4 0 0 0 0 0 4 0 4 4 4 0 4 4 4 4 4 0 0 4 0 0 0 4 0
 0 0 0 4 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 5 0 0 5 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 5 0 0 0 0 0 0 0 0 0 0]
0.4166666666666667
[[60  0  0  0  0  0]
 [ 7 47  0  3  0  3]
 [58  0  2  0  0  0]
 [53  0  0  7  0  0]
 [29  0  0  0 31  0]
 [57  0  0  0  0  3]]
              precision    recall  f1-score   support

           0       0.23      1.00      0.37        60
           1  

In [ ]:
poly_svc = svm.SVC(kernel='poly', degree=5).fit(X_train, y_train)
yhat = poly_svc.predict(X_test)
print(yhat)
print(metrics.accuracy_score(y_test, yhat))
print(confusion_matrix(y_test, yhat))
print(classification_report(y_test, yhat))

[0 0 0 0 0 0 0 0 0 0 0 0 4 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 2 0 2 0 2 2 2 2 2 1 2 0 2 0 2 2 1 2 2 1 0 2 2 1 2 1 2 2
 2 2 2 2 0 2 2 2 2 2 0 2 2 0 0 2 2 2 2 2 2 2 2 1 2 2 2 2 1 2 1 2 0 1 1 1 1
 1 3 1 1 0 1 1 3 1 1 1 4 1 0 1 0 3 3 1 0 1 1 1 0 3 1 1 3 1 3 1 3 0 1 3 1 0
 1 0 1 1 1 1 1 1 3 1 1 1 1 3 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 4 4 1 1 4 1 4 4 4 1 1 1 1 1 0 4 1 1 4 1 4 1 1 1 1 4 4 1 1 1 1 4 1
 1 1 1 4 4 1 4 1 1 1 1 1 4 1 1 1 1 1 4 4 1 1 4 1 4 4 1]
0.4777777777777778
[[58  0  1  0  1  0]
 [ 0 60  0  0  0  0]
 [ 9  8 43  0  0  0]
 [ 9 39  0 11  1  0]
 [ 0 60  0  0  0  0]
 [ 1 38  0  0 21  0]]
              precision    recall  f1-score   support

           0       0.75      0.97      0.85        60
           1  

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
for i in range(1,15) :
    poly_svc = svm.SVC(kernel='poly', degree=i).fit(X_train, y_train)
    yhat = poly_svc.predict(X_test)
    print(yhat)
    print('Degree = ',i, metrics.accuracy_score(y_test, yhat))
    # print(confusion_matrix(y_test, yhat))
    # print(classification_report(y_test, yhat))

[0 0 0 0 0 0 0 0 0 0 0 0 4 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 5 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 2 0 2 0 2 2 2 2 2 3 2 2 2 0 2 2 3 2 2 3 0 2 2 4 2 3 2 2
 2 2 2 0 0 2 2 2 2 2 0 2 0 0 2 2 2 2 2 0 2 2 2 3 2 2 2 2 3 0 3 2 0 3 1 3 3
 3 2 3 3 0 3 1 0 3 3 3 3 3 0 1 0 3 2 3 0 1 3 1 0 3 1 3 2 3 3 3 3 0 3 3 3 2
 3 2 3 3 3 3 3 3 2 1 1 3 3 3 1 3 1 3 3 4 3 3 4 4 3 4 4 4 4 4 3 3 3 3 3 3 3
 3 3 3 4 3 4 3 4 4 4 4 4 3 3 3 4 4 4 3 4 3 3 4 3 4 4 3 3 3 3 4 3 3 4 3 3 3
 4 4 4 3 4 4 4 4 4 4 4 4 4 4 4 1 4 1 4 4 4 5 4 4 4 1 4 4 4 4 4 1 4 5 1 4 4
 4 4 4 4 4 4 4 4 1 4 4 4 4 4 4 3 4 5 4 4 4 1 4 1 4 4 4]
Degree =  1 0.6222222222222222
[0 0 0 0 0 0 0 0 0 0 0 0 4 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 

In [ ]:
poly_svc = svm.SVC(kernel='poly', degree=1).fit(X_train, y_train)
yhat = poly_svc.predict(X_test)
print(yhat)
print(metrics.accuracy_score(y_test, yhat))
print(confusion_matrix(y_test, yhat))
print(classification_report(y_test, yhat))

[0 0 0 0 0 0 0 0 0 0 0 0 4 0 0 0 0 0 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 5 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 2 0 2 0 2 2 2 2 2 3 2 2 2 0 2 2 3 2 2 3 0 2 2 4 2 3 2 2
 2 2 2 0 0 2 2 2 2 2 0 2 0 0 2 2 2 2 2 0 2 2 2 3 2 2 2 2 3 0 3 2 0 3 1 3 3
 3 2 3 3 0 3 1 0 3 3 3 3 3 0 1 0 3 2 3 0 1 3 1 0 3 1 3 2 3 3 3 3 0 3 3 3 2
 3 2 3 3 3 3 3 3 2 1 1 3 3 3 1 3 1 3 3 4 3 3 4 4 3 4 4 4 4 4 3 3 3 3 3 3 3
 3 3 3 4 3 4 3 4 4 4 4 4 3 3 3 4 4 4 3 4 3 3 4 3 4 4 3 3 3 3 4 3 3 4 3 3 3
 4 4 4 3 4 4 4 4 4 4 4 4 4 4 4 1 4 1 4 4 4 5 4 4 4 1 4 4 4 4 4 1 4 5 1 4 4
 4 4 4 4 4 4 4 4 1 4 4 4 4 4 4 3 4 5 4 4 4 1 4 1 4 4 4]
0.6222222222222222
[[58  0  1  0  1  0]
 [ 0 59  0  0  0  1]
 [11  0 41  7  1  0]
 [ 8 10  6 36  0  0]
 [ 0  0  0 33 27  0]
 [ 0  8  0  1 48  3]]
              precision    recall  f1-score   support

           0       0.75      0.97      0.85        60
           1  